In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score

C:\Users\mcfal\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## EDA do CSV fornecido

In [2]:
df_failure = pd.read_csv("full_devices.csv",  encoding = "ISO-8859-1")

In [3]:
df_failure

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
124489,2015-11-02,Z1F0MA1S,0,18310224,0,0,0,10,353705,8,8,0
124490,2015-11-02,Z1F0Q8RT,0,172556680,96,107,4,11,332792,0,0,13
124491,2015-11-02,Z1F0QK05,0,19029120,4832,0,0,11,350410,0,0,0
124492,2015-11-02,Z1F0QL3N,0,226953408,0,0,0,12,358980,0,0,0


In [4]:
df_failure.device.value_counts()

device
Z1F0QLC1    304
S1F0KYCR    304
S1F0E9EP    304
S1F0EGMT    304
S1F0FGBQ    304
           ... 
W1F1CHZK      5
W1F1CJKT      5
S1F04KSC      4
W1F0WJFT      3
W1F1DA5ÿ      1
Name: count, Length: 1169, dtype: int64

In [5]:
df_failure.failure.value_counts()

failure
0    124388
1       106
Name: count, dtype: int64

In [6]:
df_failure.date.value_counts()

date
2015-01-01    1163
2015-01-02    1163
2015-01-03    1163
2015-01-04    1162
2015-01-05    1161
              ... 
2015-10-27      31
2015-10-29      31
2015-10-30      31
2015-10-31      31
2015-11-02      31
Name: count, Length: 304, dtype: int64

In [7]:
df_failure[df_failure.failure == 1].device.nunique()

106

In [8]:
df_failure.device.nunique()

1169

In [9]:
df_failure.isna().sum()

date          0
device        0
failure       0
attribute1    0
attribute2    0
attribute3    0
attribute4    0
attribute5    0
attribute6    0
attribute7    0
attribute8    0
attribute9    0
dtype: int64

In [10]:
df_failure.dtypes

date          object
device        object
failure        int64
attribute1     int64
attribute2     int64
attribute3     int64
attribute4     int64
attribute5     int64
attribute6     int64
attribute7     int64
attribute8     int64
attribute9     int64
dtype: object

In [11]:
failed_devices = df_failure[df_failure.failure == 1].device

In [12]:
failed_devices.values

array(['S1F0RRB1', 'S1F0CTDN', 'W1F0PNA5', 'W1F13SRV', 'W1F1230J',
       'W1F0T034', 'S1F0GG8X', 'S1F023H2', 'S1F0QY11', 'S1F0S2WJ',
       'W1F0Z1W9', 'W1F15S4D', 'Z1F0LVPW', 'Z1F0NVZA', 'Z1F1FCH5',
       'S1F0P3G2', 'W1F0F6BN', 'W1F0P114', 'W1F0X4FC', 'S1F0LCTV',
       'W1F03DP4', 'W1F0FW0S', 'S1F10E6M', 'S1F11MB0', 'W1F0SGHR',
       'W1F0VDH2', 'W1F0TA59', 'Z1F0LVGY', 'Z1F0MCCA', 'Z1F0P5D9',
       'W1F0NZZZ', 'W1F0T074', 'S1F0DSTY', 'S1F0TQCV', 'Z1F04GCH',
       'W1F08EDA', 'W1F1C9TE', 'S1F0S4CA', 'W1F19BPT', 'Z1F130LH',
       'S1F0GJW3', 'S1F0LD2C', 'W1F0Q8FH', 'Z1F0FSBY', 'W1F0Z4EA',
       'Z1F0QH0C', 'S1F0S4T6', 'W1F1CDDP', 'S1F0S57T', 'S1F0JD7P',
       'S1F13H80', 'Z1F148T1', 'S1F0RSZP', 'S1F0GKFX', 'S1F0LCVC',
       'W1F1BZTM', 'Z1F1RJFA', 'S1F13589', 'S1F136J0', 'S1F0F4EB',
       'W1F1C9WG', 'S1F0RR35', 'Z1F1653X', 'Z1F1AG5N', 'W1F0KCP2',
       'W1F0M35B', 'Z1F1901P', 'S1F0GKL6', 'Z1F0K451', 'W1F03D4L',
       'W1F0FKWW', 'S1F0PJJW', 'W1F0X5GW', 'S1F0L0DW', 'W1F0WB

In [13]:
#devices com falha possuem mais de uma entrada
df_failure[df_failure["device"].isin(failed_devices.values)].groupby("device").date.count() 

device
S1F023H2     19
S1F03YZM    215
S1F09DZQ    199
S1F0CTDN      7
S1F0DSTY     45
           ... 
Z1F1901P    132
Z1F1AG5N      9
Z1F1FCH5     19
Z1F1RJFA    124
Z1F1VQFY    125
Name: date, Length: 106, dtype: int64

## Dataprep
- Devemos prever no dia ANTERIOR se o aparelho terá falha
- Devemos usar as series de tempo fornecidas. 
    - Vamos assumir que teremos sempre o dado em D+0, sendo D o dia anterior ao dia da falha. Para máquinas sem falha, todos os dias fornecidos são válidos.
    - O dia da falha será um data considerado "futuro", ou seja, não teremos ele no momento anterior a falha.
    - Para cada maquina, vamos calculas medidas históricas e as medidas em D+0. Comparar ambas pode ser eficiente para a classificação.
- O modelo será visão Device, ou seja, cada device terá uma probabilidade de falha associada

In [14]:
#Pegar o dia de falha de cada device e somente utilizar dados anteriores a esse período
df_failure_date_per_device = df_failure[df_failure.failure == 1][["device","date"]].rename(columns={"date":"failure_date"})

In [15]:
df_failure_date_per_device

,device,failure_date
4885,S1F0RRB1,2015-01-05
6879,S1F0CTDN,2015-01-07
8823,W1F0PNA5,2015-01-09
11957,W1F13SRV,2015-01-13
12668,W1F1230J,2015-01-14
...,...,...
109927,W1F1CB5E,2015-08-04
114251,Z1F0MRPJ,2015-08-18
122118,S1F0JGJV,2015-10-05
122808,Z1F14BGY,2015-10-09


In [16]:
df_failure = df_failure.merge(df_failure_date_per_device, how="outer", on="device")

In [17]:
df_failure["date"] = pd.to_datetime(df_failure["date"], format='%Y-%m-%d')

In [18]:
df_failure["failure_date"] = pd.to_datetime(df_failure["failure_date"], format='%Y-%m-%d')

In [19]:
df_failure["failure_date"] = df_failure.failure_date.fillna(pd.Timestamp.max) #atribui data max para facilitar calculos

### Separa o target por device

In [20]:
target_per_device = df_failure.groupby("device")[["failure"]].max().reset_index()

In [21]:
target_per_device

,device,failure
0,S1F01085,0
1,S1F013BB,0
2,S1F0166B,0
3,S1F01E6Y,0
4,S1F01JE0,0
...,...,...
1164,Z1F1VMZB,0
1165,Z1F1VQFY,1
1166,Z1F26YZB,0
1167,Z1F282ZV,0


In [22]:
#removemos tudo que ocorre no dia da falha e depois, se houve
df_failure_to_model = df_failure[df_failure.date < df_failure.failure_date].drop("failure_date", axis=1)

In [23]:
df_failure_to_model.groupby("device").failure.max().value_counts()

failure
0    1169
Name: count, dtype: int64

In [24]:
features_to_model = ['attribute1', 'attribute2', 'attribute3',
                     'attribute4', 'attribute5', 'attribute6', 
                     'attribute7', 'attribute8','attribute9',"device"]

### Calculo de features historicas

In [25]:
hist_features = df_failure_to_model[features_to_model].groupby("device", as_index=False).agg(["min","max","mean","median"])

In [26]:
hist_features.columns = list(map('_'.join, hist_features.columns.values))

In [27]:
number_of_occurences = df_failure_to_model.groupby("device").failure.count().reset_index().rename(columns={"failure":"count"})

In [28]:
hist_features = hist_features.merge(number_of_occurences, on="device")

In [29]:
hist_features

,device,attribute1_min,attribute1_max,attribute1_mean,attribute1_median,attribute2_min,attribute2_max,attribute2_mean,attribute2_median,attribute3_min,...,attribute7_median,attribute8_min,attribute8_max,attribute8_mean,attribute8_median,attribute9_min,attribute9_max,attribute9_mean,attribute9_median,count
0,S1F01085,1650864,215630672,1.159330e+08,126045296.0,56,56,56.0,56.0,0,...,0.0,0,0,0.0,0.0,7,7,7.0,7.0,6
1,S1F013BB,14533968,243346080,1.198783e+08,100401908.0,0,0,0.0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
2,S1F0166B,7441792,224339296,1.074958e+08,79938524.0,0,0,0.0,0.0,3,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
3,S1F01E6Y,17099072,240257968,1.331127e+08,130186192.0,0,0,0.0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,48
4,S1F01JE0,79694024,235562856,1.673001e+08,171835820.0,0,0,0.0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,Z1F1VMZB,395632,242361392,1.223971e+08,121315644.0,0,0,0.0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,292
1165,Z1F1VQFY,1770608,243071840,1.233189e+08,126260080.0,0,0,0.0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,124
1166,Z1F26YZB,1574608,241938368,1.205072e+08,117770240.0,0,0,0.0,0.0,1,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,84
1167,Z1F282ZV,922256,243169296,1.180013e+08,103443936.0,0,0,0.0,0.0,1,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,84


### Calculo da feture do ultimo dia disponivel - lembrando que removemos o dia da falha

In [30]:
max_date_per_device = df_failure_to_model.groupby("device").date.max().reset_index().rename(columns={"date":"last_date"})

In [31]:
df_last_day = df_failure_to_model.merge(max_date_per_device, on="device")

In [32]:
df_last_day = df_last_day[df_last_day.date == df_last_day.last_date][features_to_model]

In [33]:
df_last_day

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,device
5,128832128,56,0,52,6,409404,0,0,7,S1F01085
11,7441792,0,3,0,6,404786,0,0,0,S1F0166B
59,147350000,0,0,0,12,259491,0,0,0,S1F01E6Y
65,185424928,0,0,0,6,412151,0,0,0,S1F01JE0
288,45858720,0,0,0,19,344875,0,0,3,S1F01R2B
...,...,...,...,...,...,...,...,...,...,...
124321,115676688,0,0,0,5,689161,0,0,0,S1F013BB
124327,165689192,0,0,0,8,660135,0,0,0,S1F02W1L
124333,14443880,0,0,0,8,687901,0,0,0,S1F02XLX
124339,96638896,0,0,0,8,684180,0,0,0,S1F03499


### Junta os dois df de features e o df de target

In [34]:
df_features = df_last_day.merge(hist_features, on="device")

In [35]:
df_features

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,device,...,attribute7_median,attribute8_min,attribute8_max,attribute8_mean,attribute8_median,attribute9_min,attribute9_max,attribute9_mean,attribute9_median,count
0,128832128,56,0,52,6,409404,0,0,7,S1F01085,...,0.0,0,0,0.0,0.0,7,7,7.0,7.0,6
1,7441792,0,3,0,6,404786,0,0,0,S1F0166B,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
2,147350000,0,0,0,12,259491,0,0,0,S1F01E6Y,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,48
3,185424928,0,0,0,6,412151,0,0,0,S1F01JE0,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
4,45858720,0,0,0,19,344875,0,0,3,S1F01R2B,...,0.0,0,0,0.0,0.0,3,3,3.0,3.0,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,115676688,0,0,0,5,689161,0,0,0,S1F013BB,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
1165,165689192,0,0,0,8,660135,0,0,0,S1F02W1L,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
1166,14443880,0,0,0,8,687901,0,0,0,S1F02XLX,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6
1167,96638896,0,0,0,8,684180,0,0,0,S1F03499,...,0.0,0,0,0.0,0.0,0,0,0.0,0.0,6


In [36]:
df_features_with_target = df_features.merge(target_per_device, on="device")

In [37]:
df_features_with_target.failure.value_counts()

failure
0    1063
1     106
Name: count, dtype: int64

### Inicio da modelagem

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df_features_with_target.drop(["failure","device"],axis=1), df_features_with_target.failure)

In [39]:
print(X_train.shape, X_test.shape)

(876, 46) (293, 46)


In [40]:
print(y_train.sum(),y_test.sum())

79 27


### Modelo - XGBoost
- Estado da arte para classificação
- Consegue trabalhar bases desbalanceadas com o parametro scale_pos_weight
- Tunning no numero de estimadores, para evitar possivel overfitting
- Vamos buscar maximizar o recall (Taxa de Verdadeiro Positivo) pois entende-se que é melhor acertar o positivo e prevenir uma possível falha
- Também vamos mensurar a performance geral com AUC

In [79]:
params = {
    "n_estimators" : [3,5,10,20], #poucos estimadores, devido ao tamanho da base
    "subsample" : [0.9], #Pouca reamostragem, devido ao tamanho da base 
    "scale_pos_weight" : [len(y_train[y_train == 0])/len(y_train[y_train == 1])], #fix no desbalanceamento
    "max_depth" : [4] #profundidade 4 para evitar overfitting
}

In [80]:
xgb = XGBClassifier()
gs = GridSearchCV(estimator = xgb, param_grid = params, verbose = 3, scoring = "recall")

In [81]:
gs.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END max_depth=4, n_estimators=3, scale_pos_weight=10.08860759493671, subsample=0.9;, score=0.500 total time=   0.0s
[CV 2/5] END max_depth=4, n_estimators=3, scale_pos_weight=10.08860759493671, subsample=0.9;, score=0.800 total time=   0.0s
[CV 3/5] END max_depth=4, n_estimators=3, scale_pos_weight=10.08860759493671, subsample=0.9;, score=0.750 total time=   0.0s
[CV 4/5] END max_depth=4, n_estimators=3, scale_pos_weight=10.08860759493671, subsample=0.9;, score=0.688 total time=   0.0s
[CV 5/5] END max_depth=4, n_estimators=3, scale_pos_weight=10.08860759493671, subsample=0.9;, score=0.562 total time=   0.0s
[CV 1/5] END max_depth=4, n_estimators=5, scale_pos_weight=10.08860759493671, subsample=0.9;, score=0.625 total time=   0.0s
[CV 2/5] END max_depth=4, n_estimators=5, scale_pos_weight=10.08860759493671, subsample=0.9;, score=0.800 total time=   0.0s
[CV 3/5] END max_depth=4, n_estimators=5, scale_pos_weight=10.088

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [4], 'n_estimators': [3, 5, 10, 20],
                         'scale_pos_weight': [10.08860759493671],
                         'subsample': [0.9]},
             scoring='recall', verbose=3)

In [82]:
xgb_estimator = gs.best_estimator_

In [83]:
xgb_estimator

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [84]:
recall_score(y_train,xgb_estimator.predict(X_train))

0.9113924050632911

In [85]:
roc_auc_score(y_train,xgb_estimator.predict(X_train))

0.9387576830837159

In [86]:
feature_importanes_xgb = pd.DataFrame().from_dict({"features":X_train.columns, "importances":xgb_estimator.feature_importances_}).sort_values("importances", ascending=False)

In [87]:
feature_importanes_xgb

,features,importances
15,attribute2_mean,0.273066
34,attribute7_max,0.215742
1,attribute2,0.065018
45,count,0.057285
23,attribute4_mean,0.057217
29,attribute6_min,0.052945
6,attribute7,0.050085
3,attribute4,0.043847
10,attribute1_max,0.043628
12,attribute1_median,0.032467


### Modelo final

In [88]:
features_selected = feature_importanes_xgb[feature_importanes_xgb.importances > 0].features.values

In [89]:
xgb_estimator.fit(X_train[features_selected], y_train) #retreina o best estimator somente com features relevantes

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=5, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [90]:
recall_score(y_test,xgb_estimator.predict(X_test[features_selected]))

0.8888888888888888

In [91]:
roc_auc_score(y_test,xgb_estimator.predict(X_test[features_selected]))

0.910609857978279

### Considerações Finais
1. Foi realizada um preparo de dados utilizando as series temporais fornecidas, agregando por device e calculando métricas descritivas
2. Nosso modelo consegue atingir uma performance de 0.88 de Recall na base de Teste e 0.91 de AUC, indicando um modelo eficiente para a previsão de falha
3. Foi realizado um preparo de dados cuidando para que não houvesse vazamento de dados (premissa explciada no notebook)